In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('..')

In [2]:
from src.experiments import PMVAE_Experiment
from src.utils import load_yaml
import torch

In [3]:
hparams = load_yaml("configs/mnist_svhn/pmvae.yaml")

In [4]:
expt = PMVAE_Experiment(hparams)

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
#####################################
Total Number of Model Parameters: 2079587


In [ ]:
# Test VAE X
# Test objectives X
# Fix callbacks

In [5]:
model = expt.model

In [6]:
mnist = torch.randn(5, 1, 28, 28)
svhn = torch.randn(5, 3, 32, 32)
inputs = [mnist, svhn]

In [13]:
elbo = expt.obj(
    model,
    inputs,
    [1.0, 1.0],
    1.0
)

In [14]:
elbo.shape

torch.Size([5])

In [18]:
log_prob = expt.test_step({"data": inputs}, 1)